# Health-Care-Data-Analysis

### - Data Preprocessing -

## Load Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus']=False

In [3]:
dat = pd.read_csv('health_data.csv', encoding='utf-8')

In [4]:
df = dat.copy()

In [5]:
df.head()

기준년도  가입자일련번호  성별코드  연령대코드(5세단위)  시도코드  신장(5Cm단위)  체중(5Kg단위)  허리둘레  시력(좌)  \
0  2016        5     2            6    11      160.0       50.0  67.0    0.4   
1  2016        8     2           16    48      155.0       60.0  81.0    0.7   
2  2016       29     2            9    11      155.0       60.0  83.0    1.0   
3  2016       41     2           15    28      150.0       45.0  71.0    0.8   
4  2016       56     2           12    26      145.0       55.0  75.0    1.2   

   시력(우)  ...  LDL콜레스테롤   혈색소  요단백  혈청크레아티닌  (혈청지오티)AST  (혈청지오티)ALT  감마지티피  \
0    0.5  ...     130.0  12.8  1.0      0.7        13.0         7.0   20.0   
1    0.5  ...     108.0  13.0  1.0      0.7        23.0        14.0   14.0   
2    1.0  ...     116.0  12.7  1.0      0.8        21.0        20.0   55.0   
3    0.5  ...     135.0  13.0  1.0      0.8        23.0        13.0   18.0   
4    1.2  ...     160.0  13.9  1.0      0.7        87.0       246.0   99.0   

   흡연상태  구강검진 수검여부   치석  
0   1.0          0  NaN  
1   1.0          0  NaN  
2   1.0          0  NaN  
3   1.0          1  0.0  
4   1.0          0  NaN  

[5 rows x 28 columns]

## Data Preprocessing

#### 분석에 사용할 수 없는 변수 제거

In [7]:
df.drop('기준년도', axis=1, inplace=True) # 값이 2016년 밖에 없다
df.drop('치석', axis=1, inplace=True) # 결측치가 너무 많고 이해하기 어려운 값이 있다.

#### 이상치 제거

In [8]:
df.loc[df['허리둘레'] > 900, '허리둘레'] = np.nan # 허리둘레 900이상인 값 결측치로 변경
df.loc[df['시력(좌)'] == 9.9, '시력(좌)'] = 0; df.loc[df['시력(우)'] == 9.9, '시력(우)'] = 0 # 시력이 0인 사람을 실명으로 간주
df = df[df['LDL콜레스테롤'] <= df['총콜레스테롤']] # 총콜레스테롤 보다 LDL이 높은 경우 제거
df.loc[df['트리글리세라이드'] > 1500, '트리글리세라이드'] = np.nan # 결측치로 변경
df.loc[df['LDL콜레스테롤'] > 300, 'LDL콜레스테롤'] = np.nan # 결측치로 변경
df.loc[df['HDL콜레스테롤'] > 300, 'HDL콜레스테롤'] = np.nan # 결측치로 변경
df.loc[df['총콜레스테롤'] > 500, '총콜레스테롤'] = np.nan # 결측치로 변경

#### 결측치 제거

In [9]:
df.dropna(inplace=True)

#### 범주형 변수로 데이터 타입 수정

In [10]:
# 1,2로 되어있는 성별코드 0,1 로 변경 후 성별로 이름 변경 
df['성별코드'] = np.where(df['성별코드'] == 1, 0, 1)
df['성별'] = df['성별코드'].astype('object')
df.drop('성별코드', axis=1, inplace=True)

# 컬럼명 '연령대코드(5세단위)', '연령대'로 변경 
df['연령대'] = df['연령대코드(5세단위)'].astype('object')
df.drop('연령대코드(5세단위)', axis=1, inplace=True)

# 시도 코드
df['시도코드'] = df['시도코드'].astype('object')

# 청력
df['청력(좌)'] = df['청력(좌)'].astype('int')
df['청력(좌)'] = df['청력(좌)'].astype('object')
df['청력(우)'] = df['청력(우)'].astype('int')
df['청력(우)'] = df['청력(우)'].astype('object')

# 요단백
df['요단백'] = df['요단백'].astype('int')
df['요단백'] = df['요단백'].astype('object')

# 흡연상태
df['흡연상태'] = df['흡연상태'].astype('int')
df['흡연상태'] = df['흡연상태'].astype('object')

# 컬럼명 '구강검진 수검여부', '구강검진_수검'으로 변경
df['구강검진_수검'] = df['구강검진 수검여부'].astype('object')
df.drop('구강검진 수검여부', axis=1, inplace=True)

# 컬럼명 '신장(5Cm단위)', '신장'으로 변경
df['신장'] = df['신장(5Cm단위)'].astype('object')
df.drop('신장(5Cm단위)', axis=1, inplace=True)

# 컬러명 '체중(5Kg단위)', '체중'으로 변경
df['체중'] = df['체중(5Kg단위)'].astype('object')
df.drop('체중(5Kg단위)', axis=1, inplace=True)

In [11]:
obj_cols = df.select_dtypes('object').columns   # object 변수
obj_cols

Index(['시도코드', '청력(좌)', '청력(우)', '요단백', '흡연상태', '성별', '연령대', '구강검진_수검', '신장',
       '체중'],
      dtype='object')

### 1. 파생변수 (기초질환)

In [12]:
# 실명
# 복부비만
# 고지혈증
# 고혈압, 고혈압_k
# 총콜레스테롤_div
# 공복혈당
# 당뇨, 당뇨_k
# 당뇨_표적장기손상
# 빈혈, 빈혈_k
# whtr, whtr_k (허리둘레/키)
# 도시
# bmi, bmi_div, bmi_div_k, BMI
# 심혈관질환 관련 콜레스테롤 지수 (non-HDL, CRF, 대사증후군)
# 혈청크레아티닌_진단
# 나이
# eGFR, GFR, eGFR_진단
# 신장질환
# 심혈관질환
# 단백뇨, 단백뇨_k
# AST구분, ALT구분, 감마지티피구분, AST/ALT비율, GTP/ALT비율, 알코올성간질환여부
# 기초대사량

#### 실명

In [13]:
df['실명'] = np.where((df['시력(좌)'] == 0) | (df['시력(우)'] == 0) == True, 1, 0).astype('object')

#### 복부비만

In [14]:
df['복부비만'] = np.where(((df['허리둘레'] >= 90) & (df['성별']==0)) | ((df['허리둘레'] >= 85) & (df['성별']==1)) == True, 1, 0).astype('object')

#### 고지혈증

In [15]:
df['고지혈증']=np.where((df['총콜레스테롤'] > 230) | (df['트리글리세라이드'] >= 200) == True, 1, 0).astype('object')

#### 고혈압

In [16]:
def div_high(row):
    if row <120 :
        return 0
    elif (row >=120) & (row <140):
        return 1
    elif row >=160 :
        return 3
    else:
        return 2
    
df['고혈압']=df['수축기혈압'].apply(div_high)

def div_low(row):
    if row <80 :
        return 0
    elif (row >=80) & (row <90):
        return 1
    elif row >=100 :
        return 3
    else:
        return 2

# 정상: 0, 고혈압전단계: 1, 1단계고혈압: 2, 2단계고혈압: 3
df['고혈압'] = np.where(df['고혈압']>=df['이완기혈압'].apply(div_low), df['고혈압'], df['이완기혈압'].apply(div_low)).astype('object')


def div_higpress_k(row):
    if row ==0 :
        return '정상'
    elif row ==1 :
        return '고혈압전단계'
    elif row ==2 :
        return '1단계고혈압'
    else:
        return '2단계고혈압'

df['고혈압_k']=df['고혈압'].apply(div_higpress_k)

#### 총콜레스테롤

In [17]:
def tr(tr) :
    if tr < 200 :
        return 0 
    elif tr < 240 :
        return 1 
    else :
        return 2

In [18]:
df['총콜레스테롤_div']=df['총콜레스테롤'].apply(tr).astype('object')

#### 공복혈당

In [19]:
def tran(tr) :
    if tr < 100 :
        return 0 
    elif tr < 110 :
        return 1 
    elif tr < 126 :
        return 2
    elif tr < 140 :
        return 3
    else :
        return 4

In [20]:
df['공복혈당']=df['식전혈당(공복혈당)'].apply(tran).astype('object')

#### 당뇨

In [21]:
def trans(tr) :
    if tr < 70 :
        return 0 # 저혈당
    elif 70 <= tr < 100 :
        return 1 # 이상없음
    elif 100 <= tr < 126 :
        return 2 # 공복혈당장애
                 # 100 ~ 110 : 100 미만 대비 당뇨병 위험도가 3.17배, 110 ~ 126 : 100 미만 대비 당뇨병 위험도가 7.42배
    else :
        return 3 # 당뇨병
    
def trans_han(tr) :
    if tr < 70 :
        return "저혈당" # 저혈당
    elif 70 <= tr < 100 :
        return "정상" # 이상없음
    elif 100 <= tr < 126 :
        return "공복혈당장애" # 공복혈당장애
                 # 100 ~ 110 : 100 미만 대비 당뇨병 위험도가 3.17배, 110 ~ 126 : 100 미만 대비 당뇨병 위험도가 7.42배
    else :
        return "당뇨병" # 당뇨병


In [22]:
df['당뇨']=df['식전혈당(공복혈당)'].apply(trans).astype('object')
df['당뇨_k']=df['식전혈당(공복혈당)'].apply(trans_han)

#### 당뇨병 표적장기손상

In [23]:
df['당뇨_표적장기손상'] = np.where((df['당뇨_k'] == '당뇨병') & 
                               ((df['요단백'] >= 5) | (df['고혈압'] == 3) | (df['총콜레스테롤'] >= 310)), 1, 0).astype('object')

#### 빈혈

In [24]:
cond1 = (df['성별'] == '1')
cond2 = (df['혈색소'] < 12)
cond3 = (df['혈색소'] < 13)

df.loc[cond1&cond2, '빈혈'] = 1 # 여자 and 혈색소 < 12 : 빈혈
df.loc[~cond1&~cond2, '빈혈'] = 0 # 여자 and 혈색소 >= 12 : 빈혈 아님
df.loc[~cond1&cond3, '빈혈'] = 1 # 남자 and 혈색소 < 13 : 빈혈
df.loc[~cond1&~cond3, '빈혈'] = 0 # 남자 and 혈색소 >= 13 : 빈혈 아님 

df['빈혈']=df['빈혈'].astype('object')

In [25]:
cond1 = (df['성별'] == '1') 
cond2 = (df['혈색소'] < 12)
cond3 = (df['혈색소'] < 13)

df.loc[cond1&cond2, '빈혈_k'] = "빈혈" # 여자 and 혈색소 < 12 : 빈혈
df.loc[~cond1&~cond2, '빈혈_k'] = "빈혈 아님" # 여자 and 혈색소 >= 12 : 빈혈 아님
df.loc[~cond1&cond3, '빈혈_k'] = "빈혈" # 남자 and 혈색소 < 13 : 빈혈
df.loc[~cond1&~cond3, '빈혈_k'] = "빈혈 아님" # 남자 and 혈색소 >= 13 : 빈혈 아님 

#### 허리둘레/키

In [26]:
df['whtr'] = df['허리둘레']/df['신장']

# 저체중: -1, 정상: 0, 과체중: 1, 비만: 2
def div_whtr(row):
    if row <0.43 :
        return '-1'
    elif (row >=0.43) & (row <0.53) :
        return '0'
    elif row >=0.58 :
        return '2'
    else:
        return '1'

df['whtr']=df['whtr'].apply(div_whtr)


def div_whtr_k(row):
    if row == '-1' :
        return '저체중'
    elif row == '0' :
        return '정상'
    elif row == '1' :
        return '과체중'
    else:
        return '비만'

df['whtr_k']=df['whtr'].apply(div_whtr_k)


#### 도시

In [27]:
def trans_addr (addr) :
    if addr == 11 :
        return "서울특별시"
    elif addr == 26 :
        return "부산광역시"
    elif addr == 27 :
        return "대구광역시"
    elif addr == 28 :
        return "인천광역시"
    elif addr == 29 :
        return "광주광역시"
    elif addr == 30 :
        return "대전광역시"
    elif addr == 31 :
        return "울산광역시"
    elif addr == 36 :
        return "세종특별자치시"
    elif addr == 41 :
        return "경기도"
    elif addr == 42 :
        return "강원도"
    elif addr == 43 :
        return "충청북도"
    elif addr == 44 :
        return "충청남도"
    elif addr == 45 :
        return "전라북도"
    elif addr == 46 :
        return "전라남도"
    elif addr == 47 :
        return "경상북도"
    elif addr == 48 :
        return "경상남도"
    elif addr == 49 :
        return "제주특별자치도"
    
df['도시']=df['시도코드'].apply(trans_addr)

#### BMI

In [28]:
df['bmi']=df['체중']/(df['신장']/100) ** 2
df['bmi']=df['bmi'].astype('float64')
df['bmi'] = df['bmi'].round(1)

In [29]:
def class_bmi_div (bmi) :
    if bmi < 18.5 :
        return "저체중"
    elif 18.5 <= bmi < 23 :
        return "정상체중"
    elif 23 <= bmi < 25 :
        return "과체중"
    elif 25 <= bmi < 30 :
        return "경도비만"
    elif 30 <= bmi < 35 :
        return "중정도비만"
    elif 35 <= bmi :
        return "고도비만"
    
def class_bmi_div_num (bmi) :
    if bmi < 18.5 :
        return 0 #"저체중"
    elif 18.5 <= bmi < 23 :
        return 1 #"정상체중"
    elif 23 <= bmi < 25 :
        return 2 #"과체중"
    elif 25 <= bmi < 30 :
        return 3 #"경도비만"
    elif 30 <= bmi < 35 :
        return 4 #"중정도비만"
    elif 35 <= bmi :
        return 5 #"고도비만"

    
df['bmi_div_k']=df['bmi'].apply(class_bmi_div)
df['bmi_div']=df['bmi'].apply(class_bmi_div_num).astype('object')

In [30]:
def class_bmi_num (bmi) :
    if bmi < 25 :
        return 0
    elif bmi < 26.5 :
        return 1
    elif bmi < 28 :
        return 2
    elif bmi < 30 :
        return 3
    else :
        return 4

df['BMI']=df['bmi'].apply(class_bmi_num).astype('object')

#### 심혈관질환 관련 콜레스테롤 지수
- non-HDL, CRF, 대사증후군

In [31]:
df['non-HDL']=df['총콜레스테롤']-df['HDL콜레스테롤'] #심혈관 질환 위험도 관련

df['CRF']=df['총콜레스테롤']/df['HDL콜레스테롤'] # 심혈관 질환 위험도 관련

In [32]:
def div_d(row):
    cnt=0
    if (row['성별'] == '0')&(row['허리둘레'] >= 90) :
        cnt+=1
    elif ~(row['성별'] == '0')&(row['허리둘레'] >= 85) :
        cnt+=1
    if row['트리글리세라이드'] >= 150 :
        cnt+=1
    if (row['성별'] == '0')&(row['HDL콜레스테롤'] < 40) :
        cnt+=1
    elif ~(row['성별'] == '0')&(row['HDL콜레스테롤'] < 50) :
        cnt+=1
    if (row['수축기혈압'] >= 130) | (row['이완기혈압'] >= 85) :
        cnt+=1
    if row['식전혈당(공복혈당)'] >= 100 :
        cnt+=1
    return cnt
        
        
df['대사증후군']=df[['성별','허리둘레','트리글리세라이드','HDL콜레스테롤','수축기혈압','이완기혈압','식전혈당(공복혈당)']].apply(div_d, axis = 1)
df['대사증후군_k']=np.where(df['대사증후군']>=3, '대사증후군', '해당없음')
df['대사증후군']=np.where(df['대사증후군']>=3, '1', '0')

#### 혈청크레아티닌

In [33]:
def func_1(row) : 
    
    # 남자
    
    if row['성별'] == 0 :
    
        if row['혈청크레아티닌'] < 0.61 :
            result =  "low"
        elif row['혈청크레아티닌'] > 1.04 :
            result =  "high"
        else : 
            result =  "normal"
            
    # 여자
    
    else :
        if row['혈청크레아티닌'] < 0.47 :
            result =  "low"
        elif row['혈청크레아티닌'] > 0.79 :
            result =  "high"
        else : 
            result =  "normal"
            
    return result

In [34]:
df['혈청크레아티닌_진단'] = df[['성별','혈청크레아티닌']].apply(func_1, axis = 1)

#### 나이

In [35]:
age_list = list(df['연령대'].unique())
age_list.sort()
age_list

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [36]:
# 연령 코드 딕셔너리 생성
# 5단위 -> 10단위 변경

age_code = {5 : 20,
            6 : 20,
            7 : 30,
            8 : 30,
            9 : 40,
            10 : 40,
            11 : 50,
            12 : 50,
            13 : 60,
            14 : 60,
            15 : 70,
            16 : 70,
            17 : 80,
            18 : 80
}

def func_1(row, age_code):
    return age_code[row]

df['나이'] = df['연령대'].apply(func_1, args = (age_code,))

#### 사구체 여과율

In [37]:
def eGFR(row) :
    if row['성별'] == 1 :
        result = 175 * (row['혈청크레아티닌'] **(-1.154)) * (row['나이']**(-0.203)) * (0.742)
        
    else :
        result = 175 * (row['혈청크레아티닌'] **(-1.154)) * (row['나이']**(-0.203))
        
    return result

In [38]:
df['eGFR'] = df[['성별', '나이', '혈청크레아티닌']].apply(eGFR, axis = 1)

#### GFR(MDRD)

In [39]:
def GFR(row) :
    if row >=60 :
        return 0
    elif row >= 45:
        return 1
    else :
        return 2

In [40]:
df['GFR'] = df['eGFR'].apply(GFR).astype('object')

#### 추정 사구체여과율 진단 (eGRF)

In [41]:
def eGFR_diagnosis(row) : 
    if row >= 90.0 :
        result = '정상 신기능'
    elif row >= 60.0 :
        result = '경도 신기능 저하'
    elif row >= 30.0 :
        result = '중등도 신기능 저하'
    elif row >= 15.0 :
        result = '중증 신기능 저하'
    else : 
        result = '말기 신부전'
    return result

In [42]:
df['eGFR_진단'] = df['eGFR'].apply(eGFR_diagnosis)

#### 고혈압성 신장병

In [43]:
cond1 = (((df['혈청크레아티닌'] >= 1.5) & (df['성별'] == 0)) | ((df['혈청크레아티닌'] >= 1.4) & (df['성별'] == 1)))
cond2 = (df['요단백'] >= 5)
cond3 = (df['eGFR'] < 60) # GFR 병수가 계산중

df.loc[cond1|cond2|cond3, '신장질환'] = 1
df['신장질환'].fillna(0, inplace=True)

#### 심혈관질환 발병위험도

In [44]:
cond1 = (((df['나이'] >= 45) & (df['성별'] == 0)) | ((df['나이'] >= 55) & (df['성별'] == 1)))
cond2 = (df['흡연상태'] == 3) # 금연 성공한 사람 어떻게 처리할까?
cond3 = ((df['BMI'] >= 25) | (df['복부비만'] == 1))
cond4 = (df['당뇨_k'] == '공복혈당장애')
cond5 = (df['HDL콜레스테롤'] < 40)
cond6 = ((df['총콜레스테롤'] >= 220) | (df['LDL콜레스테롤'] >= 150) |
        (df['트리글리세라이드'] >= 200))

cond7 = (df['HDL콜레스테롤'] >= 60) # 좋은 요인

In [45]:
df['risk_factor'] = 0
df.loc[cond1, 'risk_factor'] += 1
df.loc[cond2, 'risk_factor'] += 1
df.loc[cond3, 'risk_factor'] += 1
df.loc[cond4, 'risk_factor'] += 1
df.loc[cond5, 'risk_factor'] += 1
df.loc[cond6, 'risk_factor'] += 1
df.loc[cond7, 'risk_factor'] -= 1

In [46]:
cond8 = (df['eGFR'] < 30)
cond9 = ((30 <= df['eGFR']) & (df['eGFR'] < 60))
cond10 = (df['eGFR'] >= 60)

cond1 = (((df['당뇨_k'] == '당뇨병') & (df['당뇨_표적장기손상'] == 1)) | cond8) # 최고위험
cond2 = ((((df['당뇨_k'] == '당뇨병') & (df['당뇨_표적장기손상'] == 0)) | cond9) &
        (df['고혈압_k'] == '2단계고혈압')) # 최고위험
codn3 = ((((df['당뇨_k'] == '당뇨병') & (df['당뇨_표적장기손상'] == 0)) | cond9) &
        ((df['고혈압_k'] != '2단계고혈압'))) # 고위험
cond4 = ((df['고혈압_k'] == '2단계고혈압') & (df['risk_factor'] >= 3) &
         (df['당뇨_k'] != '당뇨병') & cond10) # 고위험
cond5 = ((df['고혈압_k'] != '2단계고혈압') &
         (df['risk_factor'] >= 3) & (df['당뇨_k'] != '당뇨병') & cond10) # 중등도위험
cond6 = (((df['고혈압_k'] == '1단계고혈압') | (df['고혈압_k'] == '2단계고혈압')) &
         ((1 <= df['risk_factor']) & (df['risk_factor'] <= 2)) &
         (df['당뇨_k'] != '당뇨병') & cond10) # 중등도위험
# cond7 = (df['고혈압_k'] == '고혈압전단계') & (1 <= df['risk_factor'] <= 2) # 저위험
cond7 = ((df['고혈압_k'] == '2단계고혈압') & (df['risk_factor'] == 0) &
         (df['당뇨_k'] != '당뇨병') & cond10) # 중등도위험
# cond8 = (((df['고혈압_k'] == '1단계고혈압') | (df['고혈압_k'] == '고혈압전단계')) & (df['risk_factor'] == 0)) # 저위험
# fillna로 나머지 저위험

df.loc[cond1|cond2, '심혈관질환'] = '최고위험'
df.loc[cond3|cond4, '심혈관질환'] = '고위험'
df.loc[cond5|cond6|cond7, '심혈관질환'] = '중등도위험'
df['심혈관질환'].fillna('저위험', inplace=True)

#### 단백뇨

In [47]:
df['단백뇨'] = df['요단백']

# 일반적 기준
# 정상: 0, 미세단백뇨: 1, 단백뇨: 2
def div_yo_k(row):
    if (row ==1)|(row==2) :
        return '정상'
    elif (row ==3)|(row==4) :
        return '미세단백뇨'
    else:
        return '단백뇨'

df['단백뇨_k']=df['단백뇨'].apply(div_yo_k)

In [48]:
# 심뇌혈관질환을 위한 기준
def div_yo(row):
    if row <= 3 :
        return '0'
    elif row==4 :
        return '1'
    else:
        return '2'

df['단백뇨']=df['단백뇨'].apply(div_yo)

#### ALT

In [49]:
cond1 = (df['(혈청지오티)ALT']<=40)
cond2 = ((df['(혈청지오티)ALT']>40)&(df['(혈청지오티)ALT']<=200))
cond3 = ((df['(혈청지오티)ALT']>200)&(df['(혈청지오티)ALT']<=400))
cond4 = (df['(혈청지오티)ALT']>400)

df.loc[cond1, 'ALT구분'] = '정상'
df.loc[cond2, 'ALT구분'] = '경도'
df.loc[cond3, 'ALT구분'] = '중등도'
df.loc[cond4, 'ALT구분'] = '중증'


#### AST

In [50]:
cond1 = (df['(혈청지오티)AST']<=40)
cond2 = ((df['(혈청지오티)AST']>40)&(df['(혈청지오티)AST']<=200))
cond3 = ((df['(혈청지오티)AST']>200)&(df['(혈청지오티)AST']<=400))
cond4 = (df['(혈청지오티)AST']>400)

df.loc[cond1, 'AST구분'] = '정상'
df.loc[cond2, 'AST구분'] = '경도'
df.loc[cond3, 'AST구분'] = '중등도'
df.loc[cond4, 'AST구분'] = '중증'

#### 감마지티피

In [51]:
cond1 = (df['감마지티피']<=34)
cond2 = (df['감마지티피']>34)

df.loc[cond1, '감마지티피구분'] = '정상'
df.loc[cond2, '감마지티피구분'] = '비정상'

#### AST/ALT 비율

In [52]:
df['AST/ALT비율'] = df['(혈청지오티)AST']/df['(혈청지오티)ALT']

#### 감마지티피/ALT비율

In [53]:
df['GTP/ALT비율'] = df['감마지티피']/df['(혈청지오티)ALT']

#### 알코올성 간질환 여부

In [54]:
# AST/ALT비율 > 1
# 감마지티피/ALT > 2.5
# AST는 300을 넘지 않음
# ALT는 종종 정상
# AST/ALT비율이 2~3 이상인 경우 알코올성 간질환을 강하게 시사한다.

cond1 = ((df['AST/ALT비율']>1)&(df['GTP/ALT비율']>2.5)&(df['(혈청지오티)AST']<=300))

df.loc[cond1, '알코올성간질환여부'] = 'O'
df.loc[~cond1, '알코올성간질환여부'] = 'X'

#### 기초대사량

In [55]:
df['기초대사량'] = np.where((df['성별'] == 0) == True, 66+(13.8*df['체중'])+(5*df['신장'])-(6.8*df['나이']), 
                       655+(9.6*df['체중'])+(1.8*df['신장'])-(4.7*df['나이']))

#### 파생변수(기초질환) 완료

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69325 entries, 0 to 69999
Data columns (total 66 columns):
가입자일련번호        69325 non-null int64
시도코드           69325 non-null object
허리둘레           69325 non-null float64
시력(좌)          69325 non-null float64
시력(우)          69325 non-null float64
청력(좌)          69325 non-null object
청력(우)          69325 non-null object
수축기혈압          69325 non-null float64
이완기혈압          69325 non-null float64
식전혈당(공복혈당)     69325 non-null float64
총콜레스테롤         69325 non-null float64
트리글리세라이드       69325 non-null float64
HDL콜레스테롤       69325 non-null float64
LDL콜레스테롤       69325 non-null float64
혈색소            69325 non-null float64
요단백            69325 non-null object
혈청크레아티닌        69325 non-null float64
(혈청지오티)AST     69325 non-null float64
(혈청지오티)ALT     69325 non-null float64
감마지티피          69325 non-null float64
흡연상태           69325 non-null object
성별             69325 non-null object
연령대            69325 non-null object
구강검진_수검        69325 non-n

In [57]:
df.head()

가입자일련번호 시도코드  허리둘레  시력(좌)  시력(우) 청력(좌) 청력(우)  수축기혈압  이완기혈압  식전혈당(공복혈당)  \
0        5   11  67.0    0.4    0.5     1     1  121.0   68.0        96.0   
1        8   48  81.0    0.7    0.5     1     1  130.0   70.0       101.0   
2       29   11  83.0    1.0    1.0     1     1  108.0   65.0        87.0   
3       41   28  71.0    0.8    0.5     1     1  135.0   70.0        90.0   
4       56   26  75.0    1.2    1.2     1     1  110.0   70.0        75.0   

   ...  심혈관질환  단백뇨  단백뇨_k  ALT구분  AST구분 감마지티피구분  AST/ALT비율  GTP/ALT비율  \
0  ...    저위험    0     정상     정상     정상      정상   1.857143   2.857143   
1  ...    저위험    0     정상     정상     정상      정상   1.642857   1.000000   
2  ...    저위험    0     정상     정상     정상     비정상   1.050000   2.750000   
3  ...    저위험    0     정상     정상     정상      정상   1.769231   1.384615   
4  ...    저위험    0     정상    중등도     경도     비정상   0.353659   0.402439   

   알코올성간질환여부  기초대사량  
0          O   1329  
1          X   1181  
2          O   1322  
3          X   1028  
4          X   1209  

[5 rows x 66 columns]

### 2. 파생변수(HRA)

In [58]:
# 위험도_허리둘레
# 복합상대위험도
# 평균절대위험도
# 조정복합상대위험도
# 절대위험도
# 심뇌혈관나이
# 교정복합상대위험도
# 교정절대위험도 
# 교정심뇌혈관나이

#### 위험도 허리둘레

In [59]:
def wr(row):
    if row['성별']==0 and row['허리둘레'] < 90:
        return 0
    elif row['성별']==1 and row['허리둘레'] < 85:
        return 0
    elif row['성별']==0 and row['허리둘레']>=90:
        return 1
    elif row['성별']==1 and row['허리둘레']>=85:
        return 1
    
df['위험도_허리둘레']=df[['성별','허리둘레']].apply(wr, axis=1)

#### 복합상대위험도

In [60]:
def man_risk(row):
    risk1, risk2, risk3, risk4, risk5, risk6, risk7, risk8, risk9 = 0, 0, 0, 0, 0, 0, 0, 0, 0
    if row['성별']==0:
        if row['bmi']<25:
            risk1 = 1.00
        elif row['bmi']< 26.5:
            risk1 = 1.04
        elif row['bmi']< 28:
            risk1 = 1.15
        elif row['bmi']< 30:
            risk1 = 1.36
        elif row['bmi']>= 30:
            risk1 = 1.58
        if row['허리둘레']<90:
            risk2 = 1.00
        elif row['허리둘레']>=90:
            risk2 = 1.44
        if row['흡연상태']==1:
            risk3 = 1.00
        elif row['흡연상태']==2:
            risk3 = 1.30
        elif row['흡연상태']==3:
            risk3 = 1.60
        if row['고혈압']==0:
            risk4 = 1.00
        elif row['고혈압']==1:
            risk4 = 1.25
        elif row['고혈압']==2:
            risk4 = 1.78
        elif row['고혈압']==3:
            risk4 = 2.71
        if row['식전혈당(공복혈당)']<100:
            risk5 = 1.00
        elif row['식전혈당(공복혈당)']<110:
            risk5 = 1.04
        elif row['식전혈당(공복혈당)']<126:
            risk5 = 1.12
        elif row['식전혈당(공복혈당)']<140:
            risk5 = 1.27
        elif row['식전혈당(공복혈당)']>=140:
            risk5 = 1.75
        if row['총콜레스테롤']<200:
            risk6 = 1.00
        elif row['총콜레스테롤']<240:
            risk6 = 1.15
        elif row['총콜레스테롤']>=240:
            risk6 = 1.37
        if row['eGFR']>=60:
            risk7 = 1.00
        elif row['eGFR']>= 45:
            risk7 = 1.34
        elif row['eGFR']< 45:
            risk7 = 2.09
        if row['단백뇨']==0:
            risk8 = 1.00
        elif row['단백뇨']==1:
            risk8 = 1.69
        elif row['단백뇨']==2:
            risk8 = 2.43
        risk9 = risk7 if risk7 > risk8 else risk8
        comp_risk = 1.00 + (risk1-1.00 if risk1>1.00 else 0) +(risk2-1.00 if risk2>1.00 else 0) + (risk3-1.00 if risk3>1.00 else 0) + (risk4-1.00 if risk4>1.00 else 0)\
        + (risk5-1.00 if risk5>1.00 else 0) + (risk6-1.00 if risk6>1.00 else 0) + (risk9-1.00 if risk9>1.00 else 0) # 복합 상대위험도
        return comp_risk
    
    elif row['성별']==1:
        if row['bmi']<25:
            risk1 = 1.00
        elif row['bmi']< 26.5:
            risk1 = 1.02
        elif row['bmi']< 28:
            risk1 = 1.03
        elif row['bmi']< 30:
            risk1 = 1.15
        elif row['bmi']>= 30:
            risk1 = 1.33
        if row['허리둘레']<85:
            risk2 = 1.00
        elif row['허리둘레']>=85:
            risk2 = 1.52
        if row['흡연상태']==1:
            risk3 = 1.00
        elif row['흡연상태']==2:
            risk3 = 1.20
        elif row['흡연상태']==3:
            risk3 = 1.60
        if row['고혈압']==0:
            risk4 = 1.00
        elif row['고혈압']==1:
            risk4 = 1.43
        elif row['고혈압']==2:
            risk4 = 2.06
        elif row['고혈압']==3:
            risk4 = 3.20
        if row['식전혈당(공복혈당)']<100:
            risk5 = 1.00
        elif row['식전혈당(공복혈당)']<110:
            risk5 = 1.03
        elif row['식전혈당(공복혈당)']<126:
            risk5 = 1.14
        elif row['식전혈당(공복혈당)']<140:
            risk5 = 1.31
        elif row['식전혈당(공복혈당)']>=140:
            risk5 = 1.80
        if row['총콜레스테롤']<200:
            risk6 = 1.00
        elif row['총콜레스테롤']<240:
            risk6 = 1.07
        elif row['총콜레스테롤']>=240:
            risk6 = 1.20
        if row['eGFR']>=60:
            risk7 = 1.00
        elif row['eGFR']>= 45:
            risk7 = 1.34
        elif row['eGFR']< 45:
            risk7 = 2.09
        if row['단백뇨']==0:
            risk8 = 1.00
        elif row['단백뇨']==1:
            risk8 = 1.69
        elif row['단백뇨']==2:
            risk8 = 2.43
        risk9 = risk7 if risk7 > risk8 else risk8
        comp_risk = 1.00 + (risk1-1.00 if risk1>1.00 else 0) +(risk2-1.00 if risk2>1.00 else 0) + (risk3-1.00 if risk3>1.00 else 0) + (risk4-1.00 if risk4>1.00 else 0)\
        + (risk5-1.00 if risk5>1.00 else 0) + (risk6-1.00 if risk6>1.00 else 0) + (risk9-1.00 if risk9>1.00 else 0) # 복합 상대위험도
        return comp_risk

df['복합상대위험도']=df[['성별', 'bmi', '허리둘레', '흡연상태', '고혈압', '식전혈당(공복혈당)', '총콜레스테롤', 'eGFR', '단백뇨']].apply(man_risk, axis=1)

#### 평균절대위험도

In [61]:
man_mean_ab_risk={
    20:99, 21:138, 22:178, 23:217, 24:256, 25:295,
    26:334, 27:374, 28:449, 29:524, 30:600, 31:675,
    32:751, 33:873, 34:995, 35:1117, 36:1239, 37:1362,
    38:1557, 39:1752, 40:1947, 41:2142, 42:2337, 43:2591, 
    44:2846, 45:3101, 46:3356, 47:3611, 48:3920, 49:4229, 
    50:4539, 51:4848, 52:5158, 53:5573, 54:5988, 55:6403,
    56:6818, 57:7234, 58:7928, 59:8622, 60:9316, 61:10010,
    62:10705, 63:11462, 64:12219, 65:12976, 66:13733, 67:14491,
    68:15205, 69:15919, 70:16633, 71:17347, 72:18062, 73:18380,
    74:18698, 75:19017, 76:19335, 77:19654, 78:19972, 79:20290,
    80:20609, 81:20927, 82:21246, 83:21564, 84:21882, 85:22201, 86:22201, 87:22201
}

In [62]:
woman_mean_ab_risk={
    20:92, 21: 103, 22:115, 23:126, 24:137, 25:149, 
    26:160, 27:172, 28:211, 29:250, 30:289, 31:328, 
    32:368, 33:402, 34:437, 35:472, 36:507, 37:542, 
    38:646, 39:750, 40:854, 41:958, 42:1062, 43:1143, 
    44:1225, 45:1307, 46:1389, 47:1471, 48:1698, 49:1926, 
    50:2154, 51:2382, 52:2610, 53:2905, 54:3201, 55:3496,
    56:3792, 57:4088, 58:4720, 59:5353, 60:5986, 61:6619,
    62:7252, 63:8001, 64:8750, 65:9499, 66:10248, 67:10997,
    68:11975, 69:12953, 70:13931, 71:14909, 72:15887, 73:16307,
    74:16728, 75:17148, 76:17569, 77:17990, 78:18410, 79:18831,
    80:19251, 81:19672, 82:20093, 83:20513, 84:20934, 85:21354, 86:21354, 87:21354
}

In [63]:
def func_2(row, age):
    return age[row]
df['평균절대위험도']=np.where(df['성별']==0, df['나이'].apply(func_2, args = (man_mean_ab_risk,)), df['나이'].apply(func_2, args=(woman_mean_ab_risk,)))

#### 조정복합상대위험도

In [64]:
man_co_risk={
    20:1.83, 21:1.83, 22:1.83, 23:1.83, 24:1.83, 25:1.91,
    26:1.91, 27:1.91, 28:1.91, 29:1.91, 30:2.02, 31:2.02,
    32:2.02, 33:2.02, 34:2.02, 35:2.08, 36:2.08, 37:2.08,
    38:2.08, 39:2.08, 40:2.11, 41:2.11, 42:2.11, 43:2.11,
    44:2.11, 45:2.14, 46:2.14, 47:2.14, 48:2.14, 49:2.14,
    50:2.19, 51:2.19, 52:2.19, 53:2.19, 54:2.19, 55:2.23,
    56:2.23, 57:2.23, 58:2.23, 59:2.23, 60:2.25, 61:2.25,
    62:2.25, 63:2.25, 64:2.25, 65:2.28, 66:2.28, 67:2.28,
    68:2.28, 69:2.28, 70:2.25, 71:2.25, 72:2.25, 73:2.25,
    74:2.25, 75:2.24, 76:2.24, 77:2.24, 78:2.24, 79:2.24,
    80:2.24, 81:2.24, 82:2.24, 83:2.24, 84:2.24, 85:2.24, 86:2.24, 87:2.24
}

In [65]:
woman_co_risk={
    20:1.43, 21: 1.43, 22:1.43, 23:1.43, 24:1.43, 25:1.40,
    26:1.40, 27:1.40, 28:1.40, 29:1.40, 30:1.44, 31:1.44,
    32:1.44, 33:1.44, 34:1.44, 35:1.47, 36:1.47, 37:1.47,
    38:1.47, 39:1.47, 40:1.54, 41:1.54, 42:1.54, 43:1.54,
    44:1.54, 45:1.64, 46:1.64, 47:1.64, 48:1.64, 49:1.64,
    50:1.78, 51:1.78, 52:1.78, 53:1.78, 54:1.78, 55:1.92,
    56:1.92, 57:1.92, 58:1.92, 59:1.92, 60:2.08, 61:2.08,
    62:2.08, 63:2.08, 64:2.08, 65:2.22, 66:2.22, 67:2.22,
    68:2.22, 69:2.22, 70:2.32, 71:2.32, 72:2.32, 73:2.32,
    74:2.32, 75:2.40, 76:2.40, 77:2.40, 78:2.40, 79:2.40,
    80:2.40, 81:2.40, 82:2.40, 83:2.40, 84:2.40, 85:2.40, 86:2.40, 87:2.40
}

In [66]:
# 현재 나이 (남성/여성) 평균대비 심 뇌혈관질환 발생 위험 ex) 45세 남성 평균 대비 발생위험

df['조정복합상대위험도']=np.where(df['성별']==0, df['복합상대위험도']/df['나이'].apply(func_2, args = (man_co_risk,)), df['복합상대위험도']/df['나이'].apply(func_2, args=(woman_co_risk,)))
df['조정복합상대위험도'] = df['조정복합상대위험도'].round(3)

#### 절대위험도

In [67]:
# 향후 10년 이내에 심뇌혈관질환이 발생할 확률 (인구 10만명 당)
df['절대위험도']=df['조정복합상대위험도']*df['평균절대위험도']

#### 심뇌혈관나이

In [68]:
def find_age(row, age):
    l = []
    for i in range(len(age)):
        a = list(age.values())
        l.append(abs(a[i]-row))
        i = l.index(min(l))
    res = list(age.keys())[i]
    return res

df['심뇌혈관나이']=np.where(df['성별']==0, df['절대위험도'].apply(find_age, args = (man_mean_ab_risk,)), df['절대위험도'].apply(find_age, args=(woman_mean_ab_risk,)))

#### 교정복합상대위험도

In [69]:
def change_risk(row):
    risk1, risk2, risk3, risk4, risk5, risk6, risk7, risk8, risk9 = 0, 0, 0, 0, 0, 0, 0, 0, 0
    if row['성별']==0:
        risk1 = 1.00
        risk2 = 1.00
        
        if row['흡연상태']==1:
            risk3 = 1.00
        elif row['흡연상태']==2:
            risk3 = 1.30
        elif row['흡연상태']==3:
            risk3 = 1.30
        
        risk4 = 1.00
        risk5 = 1.00
        risk6 = 1.00
        
        if row['eGFR']>=60:
            risk7 = 1.00
        elif row['eGFR']>= 45:
            risk7 = 1.34
        elif row['eGFR']< 45:
            risk7 = 2.09
        
        if row['단백뇨']==0:
            risk8 = 1.00
        elif row['단백뇨']==1:
            risk8 = 1.69
        elif row['단백뇨']==2:
            risk8 = 2.43
        
        risk9 = risk7 if risk7 > risk8 else risk8
        
        comp_risk = 1.00 + (risk1-1.00 if risk1>1.00 else 0) +(risk2-1.00 if risk2>1.00 else 0) + (risk3-1.00 if risk3>1.00 else 0) + (risk4-1.00 if risk4>1.00 else 0)\
        + (risk5-1.00 if risk5>1.00 else 0) + (risk6-1.00 if risk6>1.00 else 0) + (risk9-1.00 if risk9>1.00 else 0) # 복합 상대위험도
        return comp_risk
    
    elif row['성별']==1:
        risk1 = 1.00
        risk2 = 1.00
        
        if row['흡연상태']==1:
            risk3 = 1.00
        elif row['흡연상태']==2:
            risk3 = 1.20
        elif row['흡연상태']==3:
            risk3 = 1.20
            
        risk4 = 1.00
        risk5 = 1.00
        risk6 = 1.00
        
        if row['eGFR']>=60:
            risk7 = 1.00
        elif row['eGFR']>= 45:
            risk7 = 1.34
        elif row['eGFR']< 45:
            risk7 = 2.09
        
        if row['단백뇨']==0:
            risk8 = 1.00
        elif row['단백뇨']==1:
            risk8 = 1.69
        elif row['단백뇨']==2:
            risk8 = 2.43
        
        risk9 = risk7 if risk7 > risk8 else risk8
        
        comp_risk = 1.00 + (risk1-1.00 if risk1>1.00 else 0) +(risk2-1.00 if risk2>1.00 else 0) + (risk3-1.00 if risk3>1.00 else 0) + (risk4-1.00 if risk4>1.00 else 0)\
        + (risk5-1.00 if risk5>1.00 else 0) + (risk6-1.00 if risk6>1.00 else 0) + (risk9-1.00 if risk9>1.00 else 0) # 교정 복합 상대위험도
        return comp_risk

df['교정복합상대위험도']=df[['성별', '흡연상태', 'eGFR', '단백뇨']].apply(change_risk, axis=1)

#### 교정절대위험도

In [70]:
# 현재 나이 (남성/여성) 평균대비 심 뇌혈관질환 발생 위험 ex) 45세 남성 평균 대비 발생위험

df['교정절대위험도']=np.where(df['성별']==0, df['교정복합상대위험도']/df['나이'].apply(func_2, args = (man_co_risk,))*df['평균절대위험도'], df['교정복합상대위험도']/df['나이'].apply(func_2, args=(woman_co_risk,))*df['평균절대위험도'])
df['교정절대위험도'] = df['교정절대위험도'].round(3)

#### 교정심뇌혈관나이

In [71]:
df['교정심뇌혈관나이']=np.where(df['성별']==0, df['교정절대위험도'].apply(find_age, args = (man_mean_ab_risk,)), df['교정절대위험도'].apply(find_age, args=(woman_mean_ab_risk,)))

#### 파생변수(HRA) 완료

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69325 entries, 0 to 69999
Data columns (total 75 columns):
가입자일련번호        69325 non-null int64
시도코드           69325 non-null object
허리둘레           69325 non-null float64
시력(좌)          69325 non-null float64
시력(우)          69325 non-null float64
청력(좌)          69325 non-null object
청력(우)          69325 non-null object
수축기혈압          69325 non-null float64
이완기혈압          69325 non-null float64
식전혈당(공복혈당)     69325 non-null float64
총콜레스테롤         69325 non-null float64
트리글리세라이드       69325 non-null float64
HDL콜레스테롤       69325 non-null float64
LDL콜레스테롤       69325 non-null float64
혈색소            69325 non-null float64
요단백            69325 non-null object
혈청크레아티닌        69325 non-null float64
(혈청지오티)AST     69325 non-null float64
(혈청지오티)ALT     69325 non-null float64
감마지티피          69325 non-null float64
흡연상태           69325 non-null object
성별             69325 non-null object
연령대            69325 non-null object
구강검진_수검        69325 non-n

In [73]:
df.head()

가입자일련번호 시도코드  허리둘레  시력(좌)  시력(우) 청력(좌) 청력(우)  수축기혈압  이완기혈압  식전혈당(공복혈당)  \
0        5   11  67.0    0.4    0.5     1     1  121.0   68.0        96.0   
1        8   48  81.0    0.7    0.5     1     1  130.0   70.0       101.0   
2       29   11  83.0    1.0    1.0     1     1  108.0   65.0        87.0   
3       41   28  71.0    0.8    0.5     1     1  135.0   70.0        90.0   
4       56   26  75.0    1.2    1.2     1     1  110.0   70.0        75.0   

   ...  기초대사량  위험도_허리둘레  복합상대위험도  평균절대위험도  조정복합상대위험도     절대위험도  심뇌혈관나이  \
0  ...   1329         0     1.50       92      1.049    96.508      20   
1  ...   1181         0     1.48    13931      0.638  8887.978      64   
2  ...   1322         0     1.09      854      0.708   604.632      38   
3  ...   1028         0     1.50    13931      0.647  9013.357      64   
4  ...   1209         0     1.22     2154      0.685  1475.490      47   

   교정복합상대위험도   교정절대위험도  교정심뇌혈관나이  
0        1.0    64.336        20  
1        1.0  6004.741        60  
2        1.0   554.545        37  
3        1.0  6004.741        60  
4        1.0  1210.112        44  

[5 rows x 75 columns]

In [74]:
# health_data 전처리 완료한 최종 file 저장

df.to_csv('health_real_fin.csv', index=False, encoding='utf-8')